# Import packages

In [13]:
import re
import collections
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk import tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
#nltk.download('stopwords')
import ast
import pandas as pd
%matplotlib widget

[nltk_data] Downloading package punkt to /home/learner/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Get filename and list of words

In [1]:
stop_words=set(stopwords.words("english"))

def get_filename_from_title(title):
    """
    Gets the filename from the title that is passed as an argument. Returns filename if the title can be found
    """
    File_100_Books = open("books_dictionary.txt", "r")
    Contents_100_Books = File_100_Books.read()
    Dictionary_100_Books = ast.literal_eval(Contents_100_Books)
    File_100_Books.close()
    #print(Dictionary_100_Books[2]['Title'])
    for i in range(len(Dictionary_100_Books)):
        if Dictionary_100_Books[i]['Title'] == title:
            return Dictionary_100_Books[i]['Filename']

def get_list_words_from_title():
    """
    Prompt asks to enter title and returns a list of data without symbols as well as the list of words for this title
    """
    #print(get_filename_from_title("War and Peace"))
    Title_entered = input("Enter title: " )
    filename = get_filename_from_title(Title_entered)
    #print(filename)
    #file = open("test.txt", "r")
    if filename is not None:
        file = open(filename, "r")
    else:
        print("Title was not found.")
        exit()  
    data = file.read()
    data_wout_symbols = re.sub(r'[^\w]', ' ', data) #remove symbols
    words = data_wout_symbols.split() #split text at space
    sentences = tokenize.sent_tokenize(data)
    return data_wout_symbols, words, sentences

def get_list_words_from_filename(filename):
    """
    Returns list of data without symbols and list of words for argument that is filename
    """
    file = open(filename, "r")
    data = file.read()
    data_wout_symbols = re.sub(r'[^\w]', ' ', data) #remove symbols
    words = data_wout_symbols.split() #split text at space
    return data_wout_symbols, words


NameError: name 'stopwords' is not defined

# Get list of words, count how often they appear and plot rank v frequency

In [7]:
def Rank_Frequency_Plot(Count_of_words,N):
    """
    Plot of rank v frequency of N most common words in text.
    Count_of_words: ordered dictionary of how often a word occurs
    N: N most common words used for figure
    """
    Most_common_words_decreasing = collections.OrderedDict(Count_of_words.most_common(N)) #most_common returns list and we convert it back into an ordered dictionary for the diagram to work
    Frequency_common_words = [(i, Count_of_words[i] / len(words) * 100.0) for i,count in Count_of_words.most_common(N)] #Compute with what percentage the words occur
    Frequency_common_words_decreasing = collections.OrderedDict(Frequency_common_words) #Make ordered dictionary of most common words and with which percentage they occur
    #print(frequency_percentage[:5]) #print the five most common words with percentage 
    axes.set_xscale('log')
    axes.set_yscale('log')
    axes.set_xlabel(r'Rank', size=15)
    axes.set_ylabel(r'Frequency', size=15)
    axes.plot(np.arange(0,N,1), Frequency_common_words_decreasing.values())

def stats_for_one_book():
    """
    Asks for a title and returns some statistics as well as a Rank v Frequency plot
    """
    data_wout_symbols, words, sentences = get_list_words_from_title()
    for i in range(len(words)): 
        #make all words lowercase
        words[i] = words[i].lower()
    Count_of_letter = collections.Counter(data_wout_symbols) #count how often each letter occurs
    Count_of_words = collections.Counter(words) #count how often each word occurs
    #print(words) #Print all words
    print('Number of words:', len(words)) #How many words are there in total in the txt file
    Count_of_letter = collections.Counter(data_wout_symbols) #count how often each letter occurs
    Count_of_words = collections.Counter(words) #count how often each word occurs
    avg_sen_length = len(sentences)/len(words)
    #print(frequency_letter) #print all letters and how oftern they occur
    print(Count_of_words.most_common(5)) #print x most common words
    print(Count_of_words.most_common()[-1]) #print word at the end of the list
    #print(Count_of_words['war']) #print frequency of given word
    print(len(Count_of_words)/len(words)) #Number of unique words/Number total words
    print('Number of sentences:', len(sentences))
    fig, axes = plt.subplots()
    Rank_Frequency_Plot(3000)
    #Rank_Frequency_Plot(len(Count_of_words))
    #list(frequency_percentage_decreasing).index('is') #Position of a given word in list of most common words
    #plt.bar(frequency_percentage_decreasing.keys(), frequency_percentage_decreasing.values())
    plt.savefig('diagram_one.png')

# Add statistics to all books in dataframe

In [ ]:
def Stats_All_Books():
    """
    Add statistics (Number of words, most common words, ratio unique words/total words) for all books in dataframe as new columns to dataframe and plot rank v frequency for all books
    """
    Books_dataframe = pd.read_csv("Books_Dataframe.csv")  #read all books from csv into panda dataframe
    #list_number_of_words = []
    #most_common_letters = []
    #most_common_words = []
    #most_diverse = []
    list_number_of_words , most_common_letters , most_common_words , most_diverse = ([], ) * 4 #Create four empty lists, Mulitplication applied to list duplicates list 4 times
    fig, axes = plt.subplots()
    for i in range(100):
        #go through all 100 books, get filename from dataframe. If the file exists, get words, make lowecase and append stats to empty lists created above. Create rank frequency  plot for 500 most common words
        #print(Books_dataframe.iloc[i,5])
        filename = Books_dataframe.iloc[i,5]
        if os.path.exists(filename):
            data_wout_symbols, words = get_list_words_from_filename(filename)
            for j in range(len(words)):
                words[j] = words[j].lower()
            Count_of_letter = collections.Counter(data_wout_symbols) #count how often each letter occurs
            Count_of_words = collections.Counter(words) #count how often each word occurs
            list_number_of_words.append(len(words))
            most_common_letters.append(Count_of_letter.most_common(2))
            most_common_words.append(list(collections.OrderedDict(Count_of_words.most_common(2)).keys()))
            most_diverse.append(len(Count_of_words)/len(words))
            Rank_Frequency_Plot(Count_of_words,500)
        else:
            list_number_of_words.append("File not found")
            most_common_words.append("File not found")
            most_common_letters.append("File not found")
            most_diverse.append(0)
    #Now add lists as new columns to dataframe 
    Books_dataframe['Total Number Words'] = list_number_of_words
    #Books_dataframe['Two Most Common Letters'] = most_common_letters
    Books_dataframe['Two Most Common Words'] = most_common_words
    Books_dataframe['Ratio of Unique and Total Words'] = most_diverse
    Books_dataframe.to_csv(r'Books_w_stats.csv', index = False)
    max_index = pd.to_numeric(Books_dataframe['Ratio of Unique and Total Words']).idxmax() #Get index of most diverse work
    print(Books_dataframe.iloc[max_index,1]) #print name of books
    plt.savefig('diagram_many.png') #save figure of rank v frequency

# Calculate average sentence length

In [ ]:
#Read a .csv file where a pandas dataframe with all the book info is stored.
Books_dataframe = pd.read_csv("Books_Dataframe.csv") 

#Get a list of all the book filenames
book_filenames = Books_dataframe["Filename"]

#Get a list of all the book titles
book_titles = Books_dataframe["Title"]

#Create an empty list to which we append the average sentence length of each book
avg_sentence_length = []

#Loop through all the books
for filename in book_filenames:
    
    #Try to load the file, if it does not exist add an average sentence length of 0 and continue
    try:
        file = open(filename, "r") #Open file
    except FileNotFoundError:
        avg_sentence_length.append(0) #Set average sentence length to 0 if file does not exist
        continue
    
    #Obtain file content as string
    data = file.read()
    
    #Obtain all the sentences using tlk tokenize
    sentences = tokenize.sent_tokenize(data)
    
    #
    data_wout_symbols = re.sub(r'[^\w]', ' ', data) #remove symbols
    words = data_wout_symbols.split() #split text at space
    
    avg_sentence_length.append(len(words)/len(sentences))

avg_sentence_length_sorted = np.sort(avg_sentence_length)    
book_titles_sorted = book_titles[np.argsort(avg_sentence_length)]    
    
fig, ax = plt.subplots(1, figsize=(10,10))
ax.bar(book_titles_sorted, avg_sentence_length_sorted)
ax.set_ylable()
plt.xticks(rotation = 'vertical')